In [151]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [152]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
# NETWORK = 'EfficientNetB7'
NETWORK = 'NASNetLarge'
# NETWORK = 'ConvNeXtBase'

METHOD = "AVG-STD"

In [153]:
# /datassd/WHOLEIMAGE_MAMIP/EXTRACTED_FEATURES
WORKING_DIRECTORY = '/datassd/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/datassd/WHOLEIMAGE_MAMIP/hecktor2022_patient_endpoint_training.csv"

FileNotFoundError: [Errno 2] No such file or directory: '/datassd/WHOLEIMAGE_MAMIP/EXTRACTED_FEATURES/ExtractedFeatures_AVG-STD/Features_NASNetLarge_MA-MIP_WHOLE-IMAGE.csv'

In [ ]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,0.106548,0.080018,0.056250,0.075605,0.186599,0.098796,0.131114,0.060159,0.140650,...,0.023970,0.118330,0.103562,0.056917,0.155335,0.082300,0.130014,0.121878,0.101405,0.187984
1,CHUM-002,0.095130,0.060994,0.138456,0.112173,0.132268,0.114303,0.136614,0.074872,0.101066,...,0.027240,0.044778,0.095613,0.173869,0.082565,0.037778,0.143207,0.059726,0.048092,0.372010
2,CHUM-006,0.116186,0.066290,0.091819,0.061044,0.197001,0.214212,0.143477,0.078066,0.114800,...,0.020059,0.248146,0.039082,0.035827,0.145318,0.088167,0.253100,0.067566,0.107068,0.374162
3,CHUM-007,0.128491,0.115100,0.058335,0.068167,0.115533,0.094686,0.124615,0.045018,0.100409,...,0.014829,0.097203,0.052672,0.041972,0.090002,0.118428,0.267572,0.030212,0.066396,0.250226
4,CHUM-008,0.085859,0.028439,0.100462,0.069188,0.186803,0.141780,0.175887,0.046114,0.200182,...,0.044090,0.315931,0.214139,0.030088,0.135118,0.128011,0.188327,0.076861,0.093769,0.145250


# Up to here

Read Outcome csv

In [ ]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [ ]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [ ]:
OD.columns[0]

'Patient_ID'

In [ ]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [ ]:
OD.columns[0]


'Patient_ID'

In [ ]:
PET_outcome_data.columns[0]

'Patient_ID'

In [ ]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,0.106548,0.080018,0.056250,0.075605,0.186599,0.098796,0.131114,0.060159,0.140650,...,0.023970,0.118330,0.103562,0.056917,0.155335,0.082300,0.130014,0.121878,0.101405,0.187984
1,CHUM-002,0.095130,0.060994,0.138456,0.112173,0.132268,0.114303,0.136614,0.074872,0.101066,...,0.027240,0.044778,0.095613,0.173869,0.082565,0.037778,0.143207,0.059726,0.048092,0.372010
2,CHUM-006,0.116186,0.066290,0.091819,0.061044,0.197001,0.214212,0.143477,0.078066,0.114800,...,0.020059,0.248146,0.039082,0.035827,0.145318,0.088167,0.253100,0.067566,0.107068,0.374162
3,CHUM-007,0.128491,0.115100,0.058335,0.068167,0.115533,0.094686,0.124615,0.045018,0.100409,...,0.014829,0.097203,0.052672,0.041972,0.090002,0.118428,0.267572,0.030212,0.066396,0.250226
4,CHUM-008,0.085859,0.028439,0.100462,0.069188,0.186803,0.141780,0.175887,0.046114,0.200182,...,0.044090,0.315931,0.214139,0.030088,0.135118,0.128011,0.188327,0.076861,0.093769,0.145250


In [ ]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [ ]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [ ]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2552,2553,2554,2555,2556,2557,2558,2559,Relapse,RFS
0,CHUM-001,0.106548,0.080018,0.056250,0.075605,0.186599,0.098796,0.131114,0.060159,0.140650,...,0.103562,0.056917,0.155335,0.082300,0.130014,0.121878,0.101405,0.187984,0,1704
1,CHUM-002,0.095130,0.060994,0.138456,0.112173,0.132268,0.114303,0.136614,0.074872,0.101066,...,0.095613,0.173869,0.082565,0.037778,0.143207,0.059726,0.048092,0.372010,1,439
2,CHUM-006,0.116186,0.066290,0.091819,0.061044,0.197001,0.214212,0.143477,0.078066,0.114800,...,0.039082,0.035827,0.145318,0.088167,0.253100,0.067566,0.107068,0.374162,0,1186
3,CHUM-007,0.128491,0.115100,0.058335,0.068167,0.115533,0.094686,0.124615,0.045018,0.100409,...,0.052672,0.041972,0.090002,0.118428,0.267572,0.030212,0.066396,0.250226,0,1702
4,CHUM-008,0.085859,0.028439,0.100462,0.069188,0.186803,0.141780,0.175887,0.046114,0.200182,...,0.214139,0.030088,0.135118,0.128011,0.188327,0.076861,0.093769,0.145250,0,1499


In [ ]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)